In [12]:
import requests

from requests import Response
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res:Response = requests.get(youbike_url)

if res.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")


from pydantic import BaseModel,Field,field_validator,RootModel
class Site (BaseModel):
    站名:str = Field(alias='sna') 
    行政區域:str = Field(alias='sarea')
    時間:str = Field(alias='mday')
    位置:str = Field(alias='ar')
    狀態:bool = Field(alias='act')
    總數量:int = Field(alias='total')
    可借:int = Field(alias='available_rent_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')     
    可還:int = Field(alias='available_return_bikes')

    @field_validator('站名',mode='before')
    @classmethod
    def get_name(cls,value:str) ->str:
         return value.split('_')[-1]   

class Youbike(RootModel):
    root:list[Site]


data = Youbike.model_validate_json(res.text)
all_sites = data.model_dump()


import pandas as pd
df = pd.DataFrame(all_sites)
df

下載成功


,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
0,捷運科技大樓站,大安區,2024-06-17 19:27:20,復興南路二段235號前,True,28,4,25.02605,121.54360,24
1,復興南路二段273號前,大安區,2024-06-17 19:24:21,復興南路二段273號西側,True,21,8,25.02565,121.54357,13
2,國北教大實小東側門,大安區,2024-06-17 19:26:22,和平東路二段96巷7號,True,16,3,25.02429,121.54124,13
3,和平公園東側,大安區,2024-06-17 19:26:21,和平東路二段118巷33號,True,11,10,25.02351,121.54282,1
4,辛亥復興路口西北側,大安區,2024-06-17 19:27:20,復興南路二段368號,True,16,6,25.02153,121.54299,10
...,...,...,...,...,...,...,...,...,...,...
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:27:14,臺大圖書館西南側,True,30,0,25.01690,121.54031,30
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:21:21,臺大霖澤館南側,True,20,1,25.01995,121.54347,19
1420,臺大獸醫館南側,臺大公館校區,2024-06-17 19:13:50,臺大獸醫系館南側,True,24,1,25.01791,121.54242,23
1421,臺大新體育館東南側,臺大公館校區,2024-06-17 19:27:19,臺大體育館東側,True,40,9,25.02112,121.53591,31


In [13]:
df[df['可借'] <= 3]

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
2,國北教大實小東側門,大安區,2024-06-17 19:26:22,和平東路二段96巷7號,True,16,3,25.024290,121.541240,13
16,和平臥龍街口,大安區,2024-06-17 19:25:22,臥龍街1號,True,15,2,25.024461,121.546705,13
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,25.020930,121.533100,5
20,第二學生活動中心,大安區,2024-06-17 19:25:22,羅斯福路四段107號(西南側),True,25,1,25.012440,121.536610,24
21,臺灣科技大學正門,大安區,2024-06-17 19:06:22,基隆路四段43號(臺灣科技大學正門旁小側門靠田徑場),True,50,2,25.014510,121.541420,48
...,...,...,...,...,...,...,...,...,...,...
1416,臺大電機二館東南側,臺大公館校區,2024-06-17 19:25:22,臺大電機二館南側,True,14,3,25.018570,121.542470,11
1417,臺大圖資系館北側,臺大公館校區,2024-06-17 19:25:22,臺大圖書資訊館北側,True,25,1,25.018040,121.539280,24
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:27:14,臺大圖書館西南側,True,30,0,25.016900,121.540310,30
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:21:21,臺大霖澤館南側,True,20,1,25.019950,121.543470,19


In [14]:
import numpy as np

less3_df = df.query('可借 <= 3')
less3_df.shape

(446, 10)

In [21]:

df2 = df.query('可借 <= 3 or 可還 >= 10')
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
0,捷運科技大樓站,大安區,2024-06-17 19:27:20,復興南路二段235號前,True,28,4,25.026050,121.543600,24
1,復興南路二段273號前,大安區,2024-06-17 19:24:21,復興南路二段273號西側,True,21,8,25.025650,121.543570,13
2,國北教大實小東側門,大安區,2024-06-17 19:26:22,和平東路二段96巷7號,True,16,3,25.024290,121.541240,13
4,辛亥復興路口西北側,大安區,2024-06-17 19:27:20,復興南路二段368號,True,16,6,25.021530,121.542990,10
16,和平臥龍街口,大安區,2024-06-17 19:25:22,臥龍街1號,True,15,2,25.024461,121.546705,13
...,...,...,...,...,...,...,...,...,...,...
1417,臺大圖資系館北側,臺大公館校區,2024-06-17 19:25:22,臺大圖書資訊館北側,True,25,1,25.018040,121.539280,24
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:27:14,臺大圖書館西南側,True,30,0,25.016900,121.540310,30
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:21:21,臺大霖澤館南側,True,20,1,25.019950,121.543470,19
1420,臺大獸醫館南側,臺大公館校區,2024-06-17 19:13:50,臺大獸醫系館南側,True,24,1,25.017910,121.542420,23


In [24]:
df3 = df.query('行政區域 == "中山區" and 行政區域 == "中山區"')
df3

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
589,通北街65巷口,中山區,2024-06-17 19:23:19,通北街/通北街65巷口(東北側),True,16,2,25.08251,121.54004,14
590,圓山風景區,中山區,2024-06-17 19:06:22,北安路/北安路77巷口(西北側),True,15,5,25.07805,121.52950,10
591,樂群二堤頂大道口,中山區,2024-06-17 19:26:22,堤頂大道二段382號(旁),True,20,7,25.08030,121.56251,13
592,劍潭社區,中山區,2024-06-17 19:21:20,通北街146巷2弄2號(對面公園),True,13,1,25.08524,121.53748,12
593,培英公園,中山區,2024-06-17 19:22:55,崇實路/大直街62巷(東南側),True,40,21,25.08258,121.54462,19
...,...,...,...,...,...,...,...,...,...,...
737,北安明水路口(自強隧道),中山區,2024-06-17 19:26:22,北安路/明水路(公車站：自強隧道),True,12,1,25.08578,121.55225,11
738,中正國小(南京東路三段48巷),中山區,2024-06-17 19:14:20,南京東路三段48巷/龍江路76巷,True,15,2,25.05126,121.53845,13
739,臺北時代廣場,中山區,2024-06-17 19:26:22,植福路308號前,True,35,7,25.08044,121.56152,28
740,吉林農安街口(西北側),中山區,2024-06-17 19:21:15,吉林路388號,True,15,15,25.06493,121.53034,0


In [27]:
n = 0
df4 = df.query('行政區域 == "中山區" and 可借 == @n ' )
df4


,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
626,捷運中山國小站(4號出口),中山區,2024-06-17 19:27:19,民權東路一段/民權東路一段71巷口,True,20,0,25.06283,121.52697,20
653,松江路77巷口,中山區,2024-06-17 19:27:20,松江路77巷口,True,10,0,25.04985,121.53312,10
662,國立臺北大學(臺北校區),中山區,2024-06-17 19:26:22,民生東路三段67號前,True,38,0,25.05794,121.54244,38
667,濱江街199巷口,中山區,2024-06-17 19:27:20,濱江街199巷(八號疏散門前),True,15,0,25.07340,121.53985,15
668,大佳社區公園,中山區,2024-06-17 16:56:15,濱江街154巷對側,True,15,0,25.07275,121.53730,15
683,松江民生路口,中山區,2024-06-17 19:26:17,松江路208號前,True,21,0,25.05765,121.53299,21
689,松江長春路口,中山區,2024-06-17 19:27:14,松江路158號前,True,16,0,25.05499,121.53295,16
733,四平松江路口,中山區,2024-06-17 19:25:16,松江路148號旁,True,18,0,25.05366,121.53282,18
735,民權東復興北路口(西北側),中山區,2024-06-17 19:12:16,民權東路三段69號前,True,20,0,25.06246,121.54323,20
